In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
cd ./disk1/colonoscopy_cbd

In [ ]:
pwd

GPU & 모듈 임포트

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

Custom DataLoader 구현

In [ ]:
class CustomDataset(Dataset):
    def readData(self):
        all_images = []
        image_files = os.walk(self.dataset_path).__next__()[2]

        for img in image_files:
            img_path = os.path.join(self.dataset_path, img)
            image = Image.open(img_path)
            if image is not None:
                all_images.append(img_path)
    
        return all_images, len(all_images)
        
    def __init__(self, dataset_path, transforms=None):
        self.dataset_path = dataset_path
        self.transforms = transforms
        self.images, self.num_images = self.readData()

    def __getitem__(self, index):
        image = self.images[index]
        image = Image.open(image)
        image = image.convert('RGB')

        if self.transforms is not None:
            image = self.transforms(image)
        
        return image

    def __len__(self):
        return self.num_images


데이터 전처리 모듈 구현

In [ ]:
def ImageProcessor(image):
    image_size = image.shape
    # print('image size: {}'.format(image_size))
    row = image_size[0]
    col = image_size[1]

    # z-axis summation
    sum_zdim = np.sum(image,axis=2)
    # print('z-axis summation shape: {}'.format(sum_zdim.shape))
    # print('z-axis summation: {}'.format(sum_zdim))

    # x-axis summation
    sum_xdim = np.sum(sum_zdim,axis=0)
    # print('x-axis summation shape: {}'.format(sum_xdim.shape))
    # print('x-axis summation shape: {}'.format(sum_xdim))

    # y-axis summation
    sum_ydim = np.sum(sum_zdim,axis=1)
    # print('y-axis summation shape: {}'.format(sum_ydim.shape))
    # print('y-axis summation shape: {}'.format(sum_ydim))
    
    x_start_th = 70000
    x_end_th = np.min(sum_xdim) + 4000
    y_start_th = 70000
    y_end_th = np.min(sum_ydim) + 4000
    x_start, x_end, y_start, y_end = 0, 0, 0, 0
    cnt = 0
    x_boundary = sum_xdim.size - 1
    y_boundary = sum_ydim.size - 1
    for idx, item in enumerate(sum_xdim):
        if item > x_start_th:
            if cnt==1: continue
            x_start = idx
            cnt = 1
        elif item < x_end_th:
            if cnt==0: continue
            x_end = idx
            cnt = 0
        elif idx == x_boundary:
            x_end = idx

    cnt = 0
    for idx, item in enumerate(sum_ydim):
        if item > y_start_th:
            if cnt==1: continue
            y_start = idx
            cnt = 1
        elif item < y_end_th:
            if cnt==0: continue
            y_end = idx
            cnt = 0
        elif idx == y_boundary:
            y_end = idx
        
    pos1 = (x_start, y_start)
    pos2 = (x_start, y_end)
    pos3 = (x_end, y_start)
    pos4 = (x_end, y_end)
    # print('pos1: {}\npos2: {}\npos3: {}\npos4: {}'.format(pos1,pos2,pos3,pos4))

    return x_start, y_start, x_end, y_end

In [ ]:
# 이미지 전처리 코드 테스트용 & 시각화
sample_dataset = CustomDataset('./HGD/')
plt.figure(figsize=(20,15))
cnt = 1
for idx,item in enumerate(sample_dataset):
    if cnt>=40: break
    image = np.array(item)
    x0,y0,x1,y1 = ImageProcessor(image)
    plt.subplot(5,8,cnt)
    plt.title(sample_dataset.images[idx])
    plt.imshow(item)
    item = item.crop((x0,y0,x1,y1))
    plt.subplot(5,8,cnt+1)
    plt.imshow(item)

    cnt += 2
    


In [ ]:
# 전처리 된 이미지로 새로운 데이터셋 형성
cancer_dataset = CustomDataset('./Cancer/')
hgd_dataset = CustomDataset('./HGD/')
lgd_dataset = CustomDataset('./LGD/')
nor_dataset = CustomDataset('./Normal/')
PATH = '../colonoscopy_cbd_prd_v2/'

for idx, item in enumerate(cancer_dataset):
    image = np.array(item)
    x0,y0,x1,y1 = ImageProcessor(image)
    item = item.crop((x0,y0,x1,y1))
    item.save(PATH+cancer_dataset.images[idx])

for idx, item in enumerate(hgd_dataset):
    image = np.array(item)
    x0,y0,x1,y1 = ImageProcessor(image)
    item = item.crop((x0,y0,x1,y1))
    item.save(PATH+hgd_dataset.images[idx])

for idx, item in enumerate(lgd_dataset):
    image = np.array(item)
    x0,y0,x1,y1 = ImageProcessor(image)
    item = item.crop((x0,y0,x1,y1))
    item.save(PATH+lgd_dataset.images[idx])

for idx, item in enumerate(nor_dataset):
    image = np.array(item)
    x0,y0,x1,y1 = ImageProcessor(image)
    item = item.crop((x0,y0,x1,y1))
    item.save(PATH+nor_dataset.images[idx])